In [1]:
# ✅ Install compatible versions of required libraries
!pip install -q scikit-learn==1.3.2 autogluon.tabular ray==2.10.0

# ✅ Restart the kernel/runtime after running this cell (for local runtime only)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.1/312.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.8/234.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 12.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.12.0 which is incompati

In [2]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from autogluon.tabular import TabularPredictor
import pandas as pd
import warnings
import os

warnings.filterwarnings('ignore')


class CFG:
    train_path = "/kaggle/input/burnout-datathon-ieeecsmuj/train.csv"
    test_path = "/kaggle/input/burnout-datathon-ieeecsmuj/test.csv"
    sample_sub_path = "/kaggle/input/burnout-datathon-ieeecsmuj/sample_submission.csv"
    val_path="/kaggle/input/burnout-datathon-ieeecsmuj/val.csv"
    
    target = 'Lap_Time_Seconds'
    n_folds = 5
    seed = 1859
    time_limit = 3600 * 2  # adjust if needed


# Load data
train = pd.read_csv(CFG.train_path)
test = pd.read_csv(CFG.test_path)
val = pd.read_csv(CFG.val_path)

train = pd.concat([train, val], ignore_index=True)

In [3]:
train[CFG.target] = train[CFG.target].astype(float)

# Create fold column (not needed by AutoGluon but useful for tracking)
kf = KFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)
for i, (_, val_idx) in enumerate(kf.split(train)):
    train.loc[val_idx, 'fold'] = i

# Initialize TabularPredictor for regression
predictor = TabularPredictor(
    label=CFG.target,
    problem_type='regression',
    eval_metric='rmse',
    verbosity=2
)

predictor.fit(
    train_data=train.drop(columns=["fold"]),
    time_limit=CFG.time_limit,
    presets='best_quality',
    ag_args_fit={
        'num_gpus': 1,
        'num_cpus': 4
    },
    # Disable stacking
    use_bag_holdout=False,     # no internal holdout set for bagging
    num_bag_folds=0,           # disables bagging
    num_stack_levels=0         # disables stacking
)

No path specified. Models will be saved in: "AutogluonModels/ag-20250614_055107"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       28.28 GB / 31.35 GB (90.2%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=0, num_bag_sets=1
Beginning AutoGluon training ... Time limit = 7200s
AutoGluon will save models to "AutogluonModels/ag-20250614_055107"
Train Data Rows:    2187493
Train Data Columns: 44
Label Column:       Lap_Time_Seconds
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    30380.12 MB
	Train

[1000]	valid_set's rmse: 9.13899
[2000]	valid_set's rmse: 7.63927
[3000]	valid_set's rmse: 6.59858
[4000]	valid_set's rmse: 5.76337
[5000]	valid_set's rmse: 5.05678
[6000]	valid_set's rmse: 4.44246
[7000]	valid_set's rmse: 3.88599
[8000]	valid_set's rmse: 3.43531
[9000]	valid_set's rmse: 3.05222
[10000]	valid_set's rmse: 2.72002


	-2.72	 = Validation score   (-root_mean_squared_error)
	1241.6s	 = Training   runtime
	7.41s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 5802.25s of the 5802.24s of remaining time.
	Training LightGBM with GPU, note that this may negatively impact model quality compared to CPU training.
[LightGBM] [Fatal] bin size 1987 cannot run on GPU


[1000]	valid_set's rmse: 7.81052
[2000]	valid_set's rmse: 6.05369
[3000]	valid_set's rmse: 4.77096
[4000]	valid_set's rmse: 3.80214
[5000]	valid_set's rmse: 2.99271
[6000]	valid_set's rmse: 2.38741
[7000]	valid_set's rmse: 1.93856
[8000]	valid_set's rmse: 1.61369
[9000]	valid_set's rmse: 1.32332
[10000]	valid_set's rmse: 1.10505


	-1.105	 = Validation score   (-root_mean_squared_error)
	1347.73s	 = Training   runtime
	7.06s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 4446.03s of the 4446.02s of remaining time.
	-1.4003	 = Validation score   (-root_mean_squared_error)
	4238.34s	 = Training   runtime
	0.31s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 207.08s of the 207.08s of remaining time.
	Training CatBoost with GPU, note that this may negatively impact model quality compared to CPU training.
	-11.4061	 = Validation score   (-root_mean_squared_error)
	42.31s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesMSE ... Training model for up to 164.75s of the 164.74s of remaining time.
	Time limit exceeded... Skipping ExtraTreesMSE.
Fitting model: NeuralNetFastAI ... Training model for up to 108.78s of the 108.78s of remaining time.
	Ran out of time, stopping training early. (Stopping on epoch 1)
		Exception occured in `A

In [4]:
predictor.leaderboard(silent=True).style.background_gradient(subset=['score_val'], cmap='RdYlGn')

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-1.038618,root_mean_squared_error,7.379686,5586.111612,0.007704,0.038600,2,True,7
1,LightGBM,-1.105046,root_mean_squared_error,7.057517,1347.732150,7.057517,1347.732150,1,True,4
2,RandomForestMSE,-1.400302,root_mean_squared_error,0.314465,4238.340862,0.314465,4238.340862,1,True,5
3,LightGBMXT,-2.720016,root_mean_squared_error,7.406538,1241.602279,7.406538,1241.602279,1,True,3
4,CatBoost,-11.406137,root_mean_squared_error,0.022169,42.306201,0.022169,42.306201,1,True,6
5,KNeighborsUnif,-12.679524,root_mean_squared_error,34.861209,5.675828,34.861209,5.675828,1,True,1
6,KNeighborsDist,-13.220800,root_mean_squared_error,32.719030,5.063935,32.719030,5.063935,1,True,2


In [5]:
cols_to_drop = "Lap_Time_Seconds"

# Drop target column from test if present
if cols_to_drop in test.columns:
    test.drop(columns=cols_to_drop, inplace=True)

# Predict with AutoGluon predictor
pred = predictor.predict(test)

# Add predictions back to test
test["Lap_Time_Seconds"] = pred

# Save submission file
submission = test[["Lap_Time_Seconds"]]
submission.to_csv("submission_autogluon.csv", index=False)
print("✅ Submission file saved as 'submission_autogluon.csv'")

✅ Submission file saved as 'submission_autogluon.csv'
